In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
import seaborn as sns

from copy import deepcopy

#Having to import from subfolders of parent directories is a nightmare
import sys
import os
current = os.path.dirname(os.path.realpath("BatterySimV3.py"))
parent = os.path.dirname(os.path.dirname(current))
sys.path.append(parent+"\Functions")

#From Functions folder
from Merge import merge
from Battery import Battery
from DPModel import DP,DP_carb, DP_both
from Logic import logic_bat
from Funcs_Logic_DP import logic_rollout, print_price_summary, logic_series_print
from MPC import MPC, MPC_carb, MPC_both, MPCModel

In [2]:
house = "h16"
merged = merge(house)
merged

,prod_h16,cons_h16,yield,SpotPriceDKK,CO2Emission
2020-12-22 00:00:00,0.0,6.3,-6.3,11.090000,60.0
2020-12-22 01:00:00,0.0,4.8,-4.8,-0.150000,58.0
2020-12-22 02:00:00,0.0,4.8,-4.8,-55.880001,65.0
2020-12-22 03:00:00,0.0,4.3,-4.3,-72.919998,67.0
2020-12-22 04:00:00,0.0,4.0,-4.0,-53.720001,76.0
...,...,...,...,...,...
2022-12-31 19:00:00,0.0,8.7,-8.7,498.329987,209.0
2022-12-31 20:00:00,0.0,6.9,-6.9,301.190002,224.0
2022-12-31 21:00:00,0.0,5.3,-5.3,110.730003,253.0
2022-12-31 22:00:00,0.0,5.6,-5.6,73.919998,272.0


## Running all models

In [3]:
#"Input" variables
using_N=False
bat_capacity=13


Start="2022-06-19 00:00:00"
if using_N:
    N = 24*7+1
    End = pd.date_range(start=Start, periods=N, freq="h")[-1]

else:
    End = "2022-06-25 23:00:00"
    N=len(pd.date_range(start=Start,end=End,freq="h"))
    
remainder = N%24

if remainder==0:
    print(f"Period is of size {N} hours = {N//24} days\nPeroid from {Start} to {End}")
elif remainder==1:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hour\nPeroid from {Start} to {End}")
else:
    print(f"Period of size {N} hours = {N//24} days and {remainder} hours\nPeroid from {Start} to {End}")

Period is of size 168 hours = 7 days
Peroid from 2022-06-19 00:00:00 to 2022-06-25 23:00:00


## No battery

In [4]:
battery_no_bat = Battery(max_capacity=0,max_charge=0)

series_no_battery = logic_rollout(merged.loc[Start:End].copy(), battery_no_bat, logic_bat)

In [5]:
print_price_summary(series_no_battery)
logic_series_print(series_no_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  802.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 42077.0 DKK
Average emissions per year is: 233.0 kg
Number of kwh purchased in the period: 456.3
Average number of kwh purchased per year: 23951.651497005987
Average number of kwh sold per year: 34334.3748502994
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.8

  111:   1.8501,  0.0670, 21.1000, 21.1000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -3.9037,548.5673, -1.4137,  8.8946
  112:   1.9569,  0.0810, 19.9000, 19.9000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -3.8943,544.6730, -1.6119,  7.2827
  113:   2.4844,  0.1070, 16.3000, 16.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -4.0495,540.6235, -1.7441,  5.5386
  114:   2.7334,  0.1520,  8.5000,  8.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -2.3234,538.3001, -1.2920,  4.2466
  115:   3.1918,  0.1530,  2.7000,  2.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.8618,537.4383, -0.4131,  3.8335
  116:   3.0797,  0.1380, -0.6000, -0.6000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.8478,539.2862,  0.0828,  3.9163
  117:   2.6703,  0.1090, -4.7000, -4.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 12.5503,551.8364,  0.5123,  4.4286
  118:   2.1749,  0.0910, -6.5000, -6.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 14.1368,565.9733,  0.5915,  5.0201
  119:  

## Simple logic

In [6]:
battery = Battery(max_capacity=bat_capacity,max_charge=7)

series_battery = logic_rollout(merged.loc[Start:End].copy(), battery, logic_bat)

In [7]:
print_price_summary(series_battery)
logic_series_print(series_battery)

The period is from 2022-06-19 00:00:00 to 2022-06-25 23:00:00
Cost for period:  619.0  DKK
Total emissions for period:  4.0  kg
Average cost per year is: 32499.0 DKK
Average emissions per year is: 206.0 kg
Number of kwh purchased in the period: 370.1
Average number of kwh purchased per year: 19426.925748502996
Average number of kwh sold per year: 29793.90179640719
hour     price    eprice   yield    surplus  buy      charge   before   degrade  after    cost     pcumsum  emis     ecumsum 
    0:   1.5404,  0.1370, -5.5000, -5.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.4721,  8.4721,  0.7535,  0.7535
    1:   1.4134,  0.1020, -5.3000, -5.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4911, 15.9632,  0.5406,  1.2941
    2:   1.2313,  0.1460, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  5.1715, 21.1347,  0.6132,  1.9073
    3:   1.0400,  0.1550, -3.8000, -3.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  3.9519, 25.0866,  0.5890,  2.4963
    4:   0.

   71:   2.4212,  0.1360, -5.2000, -2.8000,  0.0000, -2.4000,  2.4000,  2.4000,  0.0000,  6.7794,180.8645,  0.3808,  1.5936
   72:   2.0267,  0.1460, -5.7000, -5.7000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 11.5524,192.4169,  0.8322,  2.4258
   73:   1.9973,  0.1620, -4.5000, -4.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.9878,201.4046,  0.7290,  3.1548
   74:   2.0003,  0.1730, -4.0000, -4.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.0013,209.4060,  0.6920,  3.8468
   75:   1.8486,  0.1680, -4.2000, -4.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.7640,217.1700,  0.7056,  4.5524
   76:   1.8971,  0.1350, -3.3000, -3.3000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  6.2604,223.4304,  0.4455,  4.9979
   77:   2.1148,  0.1730, -3.5000, -3.5000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  7.4019,230.8322,  0.6055,  5.6034
   78:   2.5924,  0.1950, -3.2000, -3.2000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  8.2957,239.1279,  0.6240,  6.2274
   79:  

## DP

In [ ]:
DPbat = Battery(max_capacity=bat_capacity)
series_battery_DP_price = DP(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-20 00:00:00 to 2022-06-20 23:00:00
Period from 2022-06-21 00:00:00 to 2022-06-21 23:00:00
Period from 2022-06-22 00:00:00 to 2022-06-22 23:00:00


In [ ]:
print_price_summary(series_battery_DP_price)
logic_series_print(series_battery_DP_price)

In [ ]:
series_battery_DP_carb  = DP_carb(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True)

In [ ]:
print_price_summary(series_battery_DP_carb)
logic_series_print(series_battery_DP_carb)

In [ ]:
series_battery_DP_both = DP_both(Start,End,house,merged,deepcopy(DPbat),byday=True,ints=True,degrade=False,verbose=True, ratio=0.5)

In [ ]:
print_price_summary(series_battery_DP_both)
logic_series_print(series_battery_DP_both)

## MPC

In [ ]:
MPCbat = Battery(max_capacity=bat_capacity)

Mpc = MPCModel(house, 0.1, 0.0, 1, MPCbat.max_charge, MPCbat.max_capacity)
series_battery_MPC_price = MPC(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=True)

In [ ]:
print_price_summary(series_battery_MPC_price)
logic_series_print(series_battery_MPC_price)

In [ ]:
series_battery_MPC_carb = MPC_carb(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=True)

In [ ]:
print_price_summary(series_battery_MPC_carb)
logic_series_print(series_battery_MPC_carb)

In [ ]:
series_battery_MPC_both = MPC_both(Start,End,merged,deepcopy(MPCbat),Mpc,byday=True,verbose=True, ratio=0.5)

In [ ]:
print_price_summary(series_battery_MPC_both)
logic_series_print(series_battery_MPC_both)